In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wrf-python

In [ ]:
###################################
# Created by danang eko nuryanto
# email: danang.eko@bmkg.go.id
###################################
import netCDF4 as nc
import numpy as np
import glob
from wrf import getvar, ALL_TIMES, latlon_coords, to_np
import matplotlib.pyplot as plt

lo=106.6
la=-6.5
ens=['7021','7022','7027','7161','7162','7167']
rh_point=[]
for j in range(len(ens)):
  fl = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/MWO_NWP2022/wmo_nwp_testdata/'+ens[j]+'/*'))
  wrflist=[]
  for i in range(len(fl)):
      wrflist.append(nc.Dataset(fl[i]))
  # Extract the 'rh' variable for all times
  rh = getvar(wrflist, "rh", timeidx=ALL_TIMES, method="cat")
  time = getvar(wrflist, "times", timeidx=ALL_TIMES, method="cat")
  lats, lons = latlon_coords(rh)
  lon=lons[0,:]
  lat=lats[:,0]
  ix=np.where(np.abs(lon-lo) == np.nanmin(np.abs(lon-lo)))[0][0]
  iy=np.where(np.abs(lat-la) == np.nanmin(np.abs(lat-la)))[0][0]
  rh_point.append(rh[:,0,iy,ix][:])
rh_point=np.array(rh_point).T#;print(str(to_np(time)[0])[:13])
ti=[str(to_np(i))[:13] for i in time];print(ti)

plt.figure()
plt.plot(rh_point)
plt.xticks(range(9), ti, rotation=20)
#print(fh)

In [ ]:
ave_rh=np.mean(rh_point,axis=1)
std_rh=np.std(rh_point,axis=1)
x=np.arange(0,9)
plt.figure()
plt.plot(x,ave_rh,color='k')
plt.gca().fill_between(x,ave_rh,ave_rh-std_rh,color='r')
plt.gca().fill_between(x,ave_rh,ave_rh+std_rh,color='g')
plt.xticks(range(9), ti, rotation=20)

In [ ]:
plt.figure()
plt.boxplot(rh_point.T)
plt.plot(x+1,ave_rh,color='r')
plt.gca().set_xticks(range(0, len(x)+1, 5))
plt.gca().set_xticklabels(range(0, len(x)+1, 5))
plt.xticks(range(9), ti, rotation=20)

In [ ]:
!pip install cartopy

In [ ]:
###################################
# Created by danang eko nuryanto
# email: danang.eko@bmkg.go.id
###################################
import netCDF4 as nc
import numpy as np
import glob
from wrf import getvar, ALL_TIMES, latlon_coords, to_np
import cartopy.crs as crs
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt

ens=['7021','7022','7027','7161','7162','7167']
rh_all=[]
for j in range(len(ens)):
  fl = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/MWO_NWP2022/wmo_nwp_testdata/'+ens[j]+'/*'))
  wrflist=[]
  for i in range(len(fl)):
      wrflist.append(nc.Dataset(fl[i]))
  # Extract the 'rh' variable for all times
  rh = getvar(wrflist, "rh", timeidx=ALL_TIMES, method="cat")
  time = getvar(wrflist, "times", timeidx=ALL_TIMES, method="cat")
  lats, lons = latlon_coords(rh)
  lon=lons[0,:]
  lat=lats[:,0]
  rh_all.append(rh[:,0,...])
rh_all=np.array(rh_all)
ti=str(to_np(time)[-1])[:13]
rh=rh_all[:,-1,...]
print(rh.shape)

plt.figure()
fig, axes = plt.subplots(2, 3, subplot_kw={'projection': crs.PlateCarree()}, figsize=(9,6))
ax=axes.flat
contour_levels = [50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
for i in range(len(ens)):
  cs=ax[i].contourf(lon, lat, to_np(rh[i,...]), levels=contour_levels,cmap='YlGnBu', transform=crs.PlateCarree(),extend='both')
  ax[i].set_title(ens[i])
  ax[i].coastlines()
#  ax[i].set_extent([lon[0],lon[-1],lat[0],lat[-1]], crs=crs.PlateCarree())

  # Longitude labels
  ax[i].set_xticks(np.arange(105,lon[-1],5), crs=crs.PlateCarree())
  lon_formatter = cticker.LongitudeFormatter()
  ax[i].xaxis.set_major_formatter(lon_formatter)

  # Latitude labels
  ax[i].set_yticks(np.arange(-10,lat[-1],5), crs=crs.PlateCarree())
  lat_formatter = cticker.LatitudeFormatter()
  ax[i].yaxis.set_major_formatter(lat_formatter)
# Add a colorbar axis at the bottom of the graph
#cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])
cbar_ax = fig.add_axes([0.95, 0.2, 0.02, 0.6])

# Draw the colorbar
#cbar=fig.colorbar(cs, cax=cbar_ax,orientation='horizontal')
cbar=fig.colorbar(cs, cax=cbar_ax,orientation='vertical')